In [1]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH06 to sweet spot
DC_ss = 6.4352E-3
QDevil.setValue("CH06 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 38
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
SG.setValue('Frequency', 6730.6E6)
SG.setValue('Power', -11.3)

client.close()

In [3]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qm import SimulationConfig
%matplotlib qt

# Start to save data
import datetime 
import os
import pandas as pd 

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")
second  = now.strftime("%S")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)
exp_name = 'RR_spec'
qubit_name = 'LA_Q2'

f_str =exp_name + '_'+ qubit_name +'_'+ hour+ '_' + minute +'_' + second
f_name= f_str+'.csv'
j_name = f_str + '_state.json'
p_name= f_str+'.png'

# End to save data


#########################################
# Set-up the machine and get the config #
#########################################
machine = QuAM("quam_state.json")
# machine.resonators[0].f_readout += 0.02e6
# machine.resonators[0].readout_pulse_amp=0.5
config = build_config(machine)

qubit_index = 5
res_if = machine.resonators[qubit_index].f_readout - machine.resonators[i].lo

###################
# The QUA program #
###################

n_avg = 10000

cooldown_time =20_000 * u.ns

dfs = np.arange(-2e6, +2e6, 0.02e6)

with program() as resonator_spec:
    n = declare(int)
    n_st = declare_stream()
    df = declare(int)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        save(n, n_st)
        with for_(*from_array(df, dfs)):
            update_frequency(machine.resonators[qubit_index].name, df + res_if)
            measure(
                "readout"*amp(1),
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            wait(cooldown_time, machine.resonators[qubit_index].name)
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        n_st.save('iteration')
        I_st.buffer(len(dfs)).average().save("I")
        Q_st.buffer(len(dfs)).average().save("Q")

#####################################
#  Open Communication with the QOP  #
#####################################
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

#######################
# Simulate or execute #
#######################

simulate = False

if simulate:
    simulation_config = SimulationConfig(duration=1000)
    job = qmm.simulate(config, resonator_spec, simulation_config)
    job.get_simulated_samples().con1.plot()

else:
    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec)

    # Get results from QUA program
    res_handles = job.result_handles
    I_handles = res_handles.get("I")
    Q_handles = res_handles.get("Q")
    iteration_handles = res_handles.get("iteration")
    I_handles.wait_for_values(1)
    Q_handles.wait_for_values(1)
    iteration_handles.wait_for_values(1)
    # Live plotting
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [12, 8]
    interrupt_on_close(fig, job)  # Interrupts the job when closing the figure
    while job.result_handles.is_processing():
        # Fetch results
        iteration = res_handles.get("iteration").fetch_all()
        I = res_handles.get("I").fetch_all()
        Q = res_handles.get("Q").fetch_all()
        print('iteration', iteration)
        # Plot results
        plt.subplot(211)
        plt.cla()
        plt.title("Resonator spectroscopy amplitude")
        plt.plot((dfs ) / u.MHz, np.sqrt(I**2 + Q**2), ".")
        plt.xlabel("Frequency [MHz]")
        plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [a.u.]")
        plt.subplot(212)
        plt.cla()
        # detrend removes the linear increase of phase
        phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
        plt.title("Resonator spectroscopy phase")
        plt.plot((dfs ) / u.MHz, phase, ".")
        plt.xlabel("Frequency [MHz]")
        plt.ylabel("Phase [rad]")
        plt.pause(0.1)

        

    # Fetch results
    iteration = res_handles.get("iteration").fetch_all()
    I = res_handles.get("I").fetch_all()
    Q = res_handles.get("Q").fetch_all()
    print('iteration', iteration)
    # Convert I & Q to Volts
    I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
    Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
    idx = np.argmin(np.sqrt(I**2 + Q**2))
    print(f"IF offset to add to IF: {dfs[idx] / u.MHz:.6f} MHz")
    
    # 1D spectroscopy plot
    plt.clf()
    plt.subplot(211)
    plt.title("Resonator spectroscopy amplitude")
    plt.plot((dfs ) / u.MHz, np.sqrt(I**2 + Q**2), ".")
    plt.xlabel("Frequency [MHz]")
    plt.ylabel(r"$\sqrt{I^2 + Q^2}$ [V]")
    plt.axvline(x=dfs[idx] / u.MHz)
    plt.subplot(212)
    # detrend removes the linear increase of phase
    phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
    plt.title("Resonator spectroscopy phase")
    plt.plot((dfs ) / u.MHz, phase, ".")
    plt.xlabel("Frequency [MHz]")
    plt.ylabel("Phase [rad]")
    plt.axvline(x=dfs[idx] / u.MHz)
plt.tight_layout()
plt.savefig(os.path.join(tPath, p_name))

## convert everything to lists so they are easy to parse in Matlab if desired

if isinstance(dfs,np.ndarray) == True:
   dfs1=dfs.tolist()
else:  
    pass
if isinstance(I,np.ndarray) == True:
   I1=I.tolist()
else:  
    pass
if isinstance(Q,np.ndarray) == True:
   Q1=Q.tolist()
else:  
    pass

2023-08-16 11:00:22,616 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

In [14]:
data = [('Variable', 'Value'), ('Freq', dfs1 + machine.resonators[qubit_index].f_readout), ('I', I1), ('Q', Q1)]
df = pd.DataFrame(data,index=None, columns=None)
df.to_csv(os.path.join(tPath, f_name),header=False, index=False)
open(os.path.join(tPath, j_name), "w").write(open("quam_state.json").read())

3837

In [15]:
machine._save("quam_state.json", flat_data=False)

In [16]:
print(f"IF offset to add to IF: {dfs[idx] / u.MHz:.6f} MHz")

IF offset to add to IF: -0.060000 MHz
